In [1]:
from __future__ import print_function
from net.models import SAVA_test as SAVA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import argparse
import os
from tqdm import tqdm
import time

from net.network import SelfAttention, vgg_reverse, vgg, SAVANet
from net.models import Transform, SAVA_test
%load_ext autoreload
%autoreload 2

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--sava_model', type=str, default='/public/dehezhang2/github/SAVA_Official_Implementation/models/sava_training_hard/',
                    help='Directory path to a batch of content images')
parser.add_argument('--vgg_model', type=str, default='./models/vgg/vgg_normalised.pth')
parser.add_argument('--attn_model', type=str, default='./models/attention_training/attention_kernel_iter_80000.pth')

parser.add_argument('--content_image', type=str, default='./datasets/test/content/white_horse_2.jpg',
                    help='Directory path to a batch of content images')
parser.add_argument('--style_image', type=str, default='./datasets/test/void_style/chinese_horse.jpg',
                    help='Directory path to a batch of style images')
args = parser.parse_args('')

content_name = args.content_image.split('/')[-1].split('.')[0]
style_name = args.style_image.split('/')[-1].split('.')[0]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94

cuda


torch.Size([3, 512, 512])
torch.Size([3, 512, 512])


In [4]:
# loading test data

def test_transform(size = 512):
    transform = transforms.Compose([
        transforms.Resize(size=(size, size)),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize((_R_MEAN/255.0, _G_MEAN/255.0, _B_MEAN/255.0), (0.5, 0.5, 0.5))
    ])
    return transform

def test_transform_inv():
    transform = transforms.Compose([
        transforms.Normalize((0, 0, 0), (2, 2, 2)),
        transforms.Normalize((-_R_MEAN/255.0, -_G_MEAN/255.0, -_B_MEAN/255.0), (1, 1, 1))
    ])
    return transform
# scale = 1.40625
content_tf = test_transform(int(512))
style_tf = test_transform(int(512))
content_tf_inv = test_transform_inv()
content = content_tf(Image.open(args.content_image).convert('RGB'))
print(content.shape)

style = style_tf(Image.open(args.style_image).convert('RGB'))
print(style.shape)

import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

imshow(torchvision.utils.make_grid(content_tf_inv(content)))

In [5]:
imshow(torchvision.utils.make_grid(content_tf_inv(style)))

In [6]:
def state_to_device(parameter, device):
    state_dict = parameter.state_dict()
    for key in state_dict.keys():
        state_dict[key] = state_dict[key].to(device)
    parameter.cuda()

encoder = vgg
encoder.load_state_dict(torch.load(args.vgg_model))
state_to_device(encoder, device)

self_attn = SelfAttention()
self_attn.load_state_dict(torch.load(args.attn_model))
state_to_device(self_attn, device)

transformer = Transform(in_channel = 512, self_attn=self_attn)
transformer.load_state_dict(torch.load(args.sava_model + 'transformer_iter_96000.pth'))
state_to_device(transformer, device)

decoder = vgg_reverse
decoder.load_state_dict(torch.load(args.sava_model + 'decoder_iter_96000.pth'))
state_to_device(decoder, device)
model = SAVA_test(transformer=transformer, encoder=encoder, decoder=decoder)
model.cuda()

SAVA_test(
  (transformer): Transform(
    (savanet4_1): SAVANet(
      (feat_corr): Correlation(
        (f): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
        (g): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
        (softmax): Softmax(dim=-1)
      )
      (h): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
      (self_attn): SelfAttention(
        (corr): Correlation(
          (f): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
          (g): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
          (softmax): Softmax(dim=-1)
        )
        (h): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
      )
      (softmax): Softmax(dim=-1)
      (out_conv): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
    )
    (savanet5_1): SAVANet(
      (feat_corr): Correlation(
        (f): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
        (g): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
        (softmax): Softmax(dim=-1)
      )
      (h

In [7]:
content = torch.stack([content], dim = 0)
content = content.to(device)
style = torch.stack([style], dim = 0)
style = style.to(device)
output, swapped_features, [content_attn4_1, style_attn4_1, content_attn5_1, style_attn5_1] = model.transfer(content, style)
losses = model(content, style)

[running kmeans]: 0it [00:00, ?it/s]/opt/conda/conda-bld/pytorch_1591914855613/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)
[running kmeans]: 1it [00:00, 306.94it/s, center_shift=0.000004, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 335.92it/s, center_shift=0.000000, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 486.52it/s, center_shift=0.000003, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 558.79it/s, center_shift=0.000003, iteration=1, tol=0.000100]

running k-means on cuda:0..
running k-means on cuda:0..
running k-means on cuda:0..
running k-means on cuda:0..



[running kmeans]: 1it [00:00, 368.37it/s, center_shift=0.000001, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 335.30it/s, center_shift=0.000000, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 510.32it/s, center_shift=0.000002, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 571.74it/s, center_shift=0.000003, iteration=1, tol=0.000100]

running k-means on cuda:0..
running k-means on cuda:0..
running k-means on cuda:0..
running k-means on cuda:0..



[running kmeans]: 1it [00:00, 316.77it/s, center_shift=0.000001, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 436.72it/s, center_shift=0.000000, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 453.34it/s, center_shift=0.000005, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 559.69it/s, center_shift=0.000001, iteration=1, tol=0.000100]

running k-means on cuda:0..
running k-means on cuda:0..
running k-means on cuda:0..
running k-means on cuda:0..



[running kmeans]: 1it [00:00, 346.06it/s, center_shift=0.000000, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 402.37it/s, center_shift=0.000000, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 498.43it/s, center_shift=0.000002, iteration=1, tol=0.000100]
[running kmeans]: 1it [00:00, 565.73it/s, center_shift=0.000002, iteration=1, tol=0.000100]

running k-means on cuda:0..
running k-means on cuda:0..
running k-means on cuda:0..
running k-means on cuda:0..


In [8]:
def imshow_recon(img, save_dir=None):
    npimg = img.detach().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    if save_dir != None:
        torchvision.utils.save_image(img, save_dir)
    
def show_output(output, save_dir=None):
    print("Saving to " + str(save_dir))
    content_result = content_tf_inv(output.squeeze())
    imshow_recon(torchvision.utils.make_grid(content_result.cpu()), save_dir)

Saving to test_result/white_horse_2_with_chinese_horse.png


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [9]:
show_output(output, 'test_result/' + content_name + '_with_' + style_name + '.png')

In [10]:
print(losses['content'].data)

tensor(2.3378, device='cuda:0')


In [11]:
def save_attn_map(attn_map, save_dir=None):
    channel_num = attn_map.size()[1]
    mean_sal = torch.mean(attn_map, 1, False)
    mean_sal_np = mean_sal.cpu().detach().numpy()
    mean_sal_np = mean_sal_np - np.min(mean_sal_np)
    mean_sal_np = mean_sal_np * 1.0 / np.max(mean_sal_np)
    # print(mean_sal.size())
    plt.imshow(mean_sal_np[0], cmap=cm.get_cmap('rainbow', 1000))
    plt.colorbar()
    plt.axis('off')
#     plt.savefig(save_dir)

In [12]:
save_attn_map((content_attn4_1 > 0.0001).type(torch.cuda.FloatTensor))

In [13]:
save_attn_map((style_attn4_1 > 0.0001).type(torch.cuda.FloatTensor))

In [14]:
def attn_mask(x):
    x_size = x.shape
    x = x.reshape(x_size[0], x_size[1], -1)
    _, indices = x.topk(int(x.shape[2] * 0.825), dim=2, largest=False, sorted=True)
    attn = torch.ones(x.shape)
    attn[:, :, indices] = 0
    attn = attn.view(x_size).type(torch.cuda.FloatTensor)
    return attn

In [15]:
save_attn_map(attn_mask(content_attn4_1))

In [16]:
save_attn_map(attn_mask(style_attn4_1))